In [ ]:
import pydicom
import glob

dicom_files=sorted(glob.glob("/home/abdullah/BOUN/MIPA/erdem bey calısma/1.2.410.200010.20230508.145831.100041/1.2.410.200010.20230508.145831.100041.103722/*.dcm"))


volume = pydicom.Dataset()

for dicom_file in dicom_files:
    dicom = pydicom.read_file(dicom_file)
    volume.SliceThickness = dicom.SliceThickness
    volume.PixelSpacing = dicom.PixelSpacing
    volume.ImagePositionPatient = dicom.ImagePositionPatient
    volume.add_data(dicom.pixel_array)

volume.save('combined_volume.dcm')

In [ ]:
import glob
len(glob.glob("/home/abdullah/BOUN/MIPA/MIPA/*/*/AXIAL"))

In [ ]:
import pydicom
import numpy as np
import SimpleITK as sitk

def combine_dicoms(dicom_files):
    # Load DICOM files using PyDICOM
    dicom_slices = [pydicom.dcmread(file) for file in dicom_files]

    # Sort the slices based on their position
    dicom_slices.sort(key=lambda x: float(x.ImagePositionPatient[2]))

    # Extract necessary metadata from the DICOM files
    spacing = np.array([float(dicom_slices[0].PixelSpacing[0]), float(dicom_slices[0].PixelSpacing[1]), float(dicom_slices[0].SliceThickness)])
    dimensions = (int(dicom_slices[0].Rows), int(dicom_slices[0].Columns), len(dicom_slices))

    # Create an empty 3D numpy array to hold the volume
    volume = np.zeros(dimensions, dtype=dicom_slices[0].pixel_array.dtype)

    # Fill the volume with DICOM pixel data
    for i, dicom_slice in enumerate(dicom_slices):
        volume[:, :, i] = dicom_slice.pixel_array

    # Convert the volume to a SimpleITK image
    sitk_image = sitk.GetImageFromArray(volume)
    sitk_image.SetSpacing(spacing)

    return sitk_image,dicom_slices

# Provide a list of DICOM file paths

# Combine DICOMs into a volume
volume,dicom_slices = combine_dicoms(dicom_files)

# You now have a volume that you can use for further processing or visualization


In [ ]:
output_dicom = pydicom.FileDataset("output.dcm", dicom_slices[0], file_meta=dicom_slices[0].file_meta, preamble=b"\0" * 128)

# Set the necessary DICOM attributes for the new file
output_dicom.Modality = "CT"  # Set the modality
output_dicom.PatientName = "John Doe"  # Set patient name
# Add other relevant DICOM attributes here

# Convert the volume to a 16-bit pixel array
pixel_array = volume.astype(np.uint16)

# Set the pixel array in the DICOM dataset
output_dicom.PixelData = pixel_array.tobytes()

# Set the dimensions of the volume
output_dicom.Rows, output_dicom.Columns = pixel_array.shape[1], pixel_array.shape[2]

# Set other necessary DICOM attributes related to pixel data and geometry

# Save the DICOM file
output_dicom.save_as("output.dcm")

In [ ]:
 dcm2niix -b y -o . /home/abdullah/BOUN/MIPA/"erdem bey calısma"/1.2.410.200010.20230508.145831.100041/1.2.410.200010.20230508.145831.100041.103722/*.dcm

In [ ]:
import glob 
import subprocess
import pydicom
import os
from tqdm import tqdm
root_paths=glob.glob("/home/abdullah/BOUN/MIPA/MIPA/*")
images=[]
for root_path in tqdm(root_paths,total=len(root_paths)):
    paths=sorted(glob.glob(root_path+"/*"))
    for path in paths:
            try:
                  dicoms=sorted(glob.glob(path+"/*.dcm"))
                  dicom_file=pydicom.read_file(dicoms[0])
                  
                  #dom=check_dominant_axis(dicoms[0])
                  dom=dicom_file.ImageType[-1]
                  try:
                        os.mkdir(f"{path}/{dom}")
                  except:
                        pass
                  command=f"dcm2niix  -t y  -s n -m y -ba n -b y -o {path}/{dom} {path}/*.dcm"
                  output = os.system(command)
            except:
                  pass
        



In [ ]:
dicom_file

In [ ]:
""" USAGE

Author: Miguel Rueda
e-mail: makquel@gmail.com

"""

import numpy as np
import SimpleITK as sitk
import gc
import sys
import os
import logging
from color_conversion import cv2_grey_to_color
from dataclasses import dataclass

logger = logging.getLogger("pre_process")


@dataclass
class image_preparation:
    r""" """

    def __init__(
        self,
        image_path,
        window_level=-600,
        window_width=1500,
        cormack_level=False,
        image_window=True,
        image_size=[256, 256, 256],
    ):

        self.raw_image = image_path
        self.window_level = window_level
        self.window_width = window_width
        self.cormack_level = cormack_level
        if self.cormack_level:
            self.window_level = 0
            self.window_width = 2000

        self.image_window = image_window
        self.image_size = image_size

        try:
            self.image = self.loadImage()
        except:
            logger.error("File {} not found or not supported!".format(self.raw_image))

    def __del__(self):
        logger.info("Image pre-process destructor Called!")

    @staticmethod
    def _prepare_size(image_path, downsample_rate):
        Size = image_path.GetSize() / downsample_rate
        return Size

    def getImage(self):
        """
        Returns the image in the SimpleITK object format.
        """
        return self.image

    def setWindowLevel(self, value):
        """
        Sets the minimum window value.
        """
        self.window_level = value

    def setWindowWidth(self, value):
        """
        Sets the maximum window value.
        """
        self.window_width = value

    def setScaleCormack(self, flag):
        """
        Sets the Boolean value indicating whether or not the Hounsfield scale
        conversion to Cormack level will be performed.
        """
        self.cormack_level = flag

    def setImageWindow(self, flag):
        """
        Sets the Boolean value indicating whether or not the image will be
        windowed.
        """
        self.image_window = flag

    def hounsfield_to_cormack(self, image) -> sitk.Image:
        r"""Conversion formula suggested by Chris Rorden
        in matlab's clinical toolbox
        https://www.nitrc.org/projects/clinicaltbx/

        Parameters
        ----------
        image : SimpleITK image object
            Raw CT scan (Hounsfield scale)
        Returns
        -------
        type:
            SimpleITK image object.
        describe :
            Cormack scaled CT scan
        """
        img_data = sitk.GetArrayFromImage(image)
        t = img_data.flatten()
        t1 = np.zeros(t.size)
        t1[np.where(t > 100)] = t[np.where(t > 100)] + 3000
        t1[np.where(np.logical_and(t >= -1000, t <= -100))] = (
            t[np.where(np.logical_and(t >= -1000, t <= -100))] + 1000
        )
        t1[np.where(np.logical_and(t >= -99, t <= 100))] = (
            t[np.where(np.logical_and(t >= -99, t <= 100))] + 99
        ) * 11 + 911
        trans_img = t1.reshape(img_data.shape)

        res_img = sitk.GetImageFromArray(trans_img)
        res_img.CopyInformation(image)

        return res_img

    def imageWindow(self, image) -> sitk.Image:
        r"""A one-line summary that does not use variable names or the
        function name.
        Several sentences providing an extended description. Refer to
        variables using back-ticks, e.g. `var`.

        Parameters
        ----------
        image : array_like
            Array_like means all those objects -- lists, nested lists, etc. --
            that can be converted to an array.  We can also refer to
            variables like `var1`.
        Returns
        -------
        type
            Explanation of anonymous return value of type ``type``.
        describe : type
            Explanation of return value named `describe`.
        out : type
            Explanation of `out`.
        Other Parameters
        ----------------
        only_seldom_used_keywords : type
            Explanation
        common_parameters_listed_above : type
            Explanation
        """
        if self.cormack_level:
            image = self.hounsfield_to_cormack(image)
            logger.info("Cormack level selected")

        windowing = sitk.IntensityWindowingImageFilter()
        windowing.SetWindowMinimum(self.window_level)
        windowing.SetWindowMaximum(self.window_width)
        img_win = windowing.Execute(image)

        return img_win

    def loadImage(self) -> sitk.Image:
        """ """
        # Reads the image using SimpleITK
        image = sitk.ReadImage(self.raw_image, imageIO="NiftiImageIO")
        # Performs linear windowing
        if self.image_window:
            ct_image = self.imageWindow(image)
        else:
            ct_image = image
        # ct_array = sitk.GetArrayFromImage(itkimage)
        # Performs downsampling for Deep Learning purpouse
        ct_image = self.downsampleImage(ct_image)
        logger.info(str(ct_image.GetSize()))

        return ct_image

    def downsampleImage(self, image) -> sitk.Image:
        r"""Downsample funtion for deep learning preprocess purpose image
        reference_size: size in vector like format (i.e. [sx, sy,sz])
        Parameters
        ----------
        image : SimpleITK image object
            Windowed CT scan
        Returns
        -------
        type:
            SimpleITK image object.
        describe :
            Downsampled CT scan
        """
        original_CT = image
        # NIfTi(RAS) to ITK(LPS)
        original_CT = sitk.DICOMOrient(original_CT, "LPS")
        dimension = original_CT.GetDimension()
        reference_physical_size = np.zeros(original_CT.GetDimension())
        reference_physical_size[:] = [
            (sz - 1) * spc if sz * spc > max_ else max_
            for sz, spc, max_ in zip(
                original_CT.GetSize(), original_CT.GetSpacing(), reference_physical_size
            )
        ]

        reference_origin = original_CT.GetOrigin()
        reference_direction = original_CT.GetDirection()
        # NOTE: Looks like the downsampled image is mirrored over the y axis
        #     reference_direction = [1.,0.,0.,0.,1.,0.,0.,0.,1.]
        reference_size = self.image_size
        logger.info(str(reference_size))
        reference_spacing = [
            phys_sz / (sz - 1)
            for sz, phys_sz in zip(reference_size, reference_physical_size)
        ]

        reference_image = sitk.Image(reference_size, original_CT.GetPixelIDValue())
        reference_image.SetOrigin(reference_origin)
        reference_image.SetSpacing(reference_spacing)
        reference_image.SetDirection(reference_direction)

        reference_center = np.array(
            reference_image.TransformContinuousIndexToPhysicalPoint(
                np.array(reference_image.GetSize()) / 2.0
            )
        )

        transform = sitk.AffineTransform(dimension)
        transform.SetMatrix(original_CT.GetDirection())
        transform.SetTranslation(np.array(original_CT.GetOrigin()) - reference_origin)
        # Modify the transformation to align the centers of the original and
        # reference image instead of their origins.
        centering_transform = sitk.TranslationTransform(dimension)
        img_center = np.array(
            original_CT.TransformContinuousIndexToPhysicalPoint(
                np.array(original_CT.GetSize()) / 2.0
            )
        )
        centering_transform.SetOffset(
            np.array(
                transform.GetInverse().TransformPoint(img_center) - reference_center
            )
        )
        centered_transform = sitk.CompositeTransform([transform, centering_transform])

        return sitk.Resample(
            original_CT, reference_image, centered_transform, sitk.sitkLinear, 0.0
        )

    def gantryRemoval(self, ct_image) -> sitk.Image:
        r"""Intensity based tight crop

        Parameters
        ----------
        ct_image : SimpleITK image object
            Raw CT scan (Hounsfield scale)
        Returns
        -------
        type:
            SimpleITK image object.
        describe :
            tight cropped CT scan
        """
        max_hu = int(np.max(sitk.GetArrayFromImage(ct_image)))
        min_hu = int(np.min(sitk.GetArrayFromImage(ct_image)))
        # binary mask of chest + artifact(s)
        image_thr = sitk.BinaryThreshold(ct_image, -600, max_hu)
        # label image with connected components
        cc = sitk.ConnectedComponent(image_thr)
        # reorder labels with largest components first
        cc = sitk.RelabelComponent(cc)
        # get the first component (chest)
        cc = sitk.BinaryThreshold(cc, 1, 1)
        #  erode and dilate to denoise
        chestMask = sitk.BinaryMorphologicalOpening(cc, [2] * 3)
        # apply mask on image and return
        tmp_img = sitk.Mask(ct_image, chestMask, min_hu)

        return tmp_img

In [ ]:
image_ct_brain=image_preparation("1.2.410.200010.20230508.145831.100041/1.2.410.200010.20230508.145831.100041.124908/AXIAL/1.2.410.200010.20230508.145831.100041.124908_Beyin+Boyun_Anjio_Head_20230504005400_702.nii",window_level=40,window_width=80)
image_ct_bone=image_preparation("1.2.410.200010.20230508.145831.100041.103722_Beyin+Boyun_Anjio_Head_20230504005400_703.nii",window_level=600,window_width=2800)

In [ ]:
image_ct_brain.setImageWindow(True)
image_gantry=image_ct_brain.gantryRemoval(image_ct_brain.image)
image_gantry

In [ ]:
import nibabel as nib

In [ ]:
affine = np.eye(4)
nifti_image = nib.Nifti1Image(sitk.GetArrayFromImage(image_gantry), affine)


In [ ]:
import time
import numpy as np

from skimage import io

vol = sitk.GetArrayFromImage(image_gantry)

vol=vol[:,:,65:150].T
r, c = volume[0].shape

# Define frames
import plotly.graph_objects as go
nb_frames = 67

fig = go.Figure(frames=[go.Frame(data=go.Surface(
    z=(6.7 - k * 0.1) * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67 - k]),
    cmin=0, cmax=200
    ),
    name=str(k) # you need to name the frame for the animation to behave properly
    )
    for k in range(nb_frames)])

# Add data to be displayed before animation starts
fig.add_trace(go.Surface(
    z=6.7 * np.ones((r, c)),
    surfacecolor=np.flipud(volume[67]),
    colorscale='Gray',
    cmin=0, cmax=200,
    colorbar=dict(thickness=20, ticklen=4)
    ))


def frame_args(duration):
    return {
            "frame": {"duration": duration},
            "mode": "immediate",
            "fromcurrent": True,
            "transition": {"duration": duration, "easing": "linear"},
        }

sliders = [
            {
                "pad": {"b": 10, "t": 60},
                "len": 0.9,
                "x": 0.1,
                "y": 0,
                "steps": [
                    {
                        "args": [[f.name], frame_args(0)],
                        "label": str(k),
                        "method": "animate",
                    }
                    for k, f in enumerate(fig.frames)
                ],
            }
        ]

# Layout
fig.update_layout(
         title='Slices in volumetric data',
         width=600,
         height=600,
         scene=dict(
                    zaxis=dict(range=[-0.1, 6.8], autorange=False),
                    aspectratio=dict(x=1, y=1, z=1),
                    ),
         updatemenus = [
            {
                "buttons": [
                    {
                        "args": [None, frame_args(50)],
                        "label": "&#9654;", # play symbol
                        "method": "animate",
                    },
                    {
                        "args": [[None], frame_args(0)],
                        "label": "&#9724;", # pause symbol
                        "method": "animate",
                    },
                ],
                "direction": "up",
                "pad": {"r": 10, "t": 70},
                "type": "buttons",
                "x": 0.1,
                "y": 0,
            }
         ],
         sliders=sliders
)

fig.show()

In [ ]:
import SimpleITK as sitk
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
image_array = sitk.GetArrayFromImage(image_gantry)



In [ ]:
# Get the dimensions of the image
size_x, size_y, size_z = image_array.shape

# Create a meshgrid for the x, y, z coordinates
x = range(size_x)
y = range(size_y)
z = range(size_z)
X, Y, Z = np.meshgrid(x, y, z)

# Create a figure and a 3D subplot
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Plot the 3D image
ax.scatter(X.flatten(), Y.flatten(), Z.flatten(), c=image_array.flatten(), cmap='gray')

# Set the axis labels
ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# Show the plot
plt.show()

In [ ]:
plt.figure(figsize=(26,10))
plt.imshow(image_array[45,:,:].squeeze(),cmap='gray')

In [ ]:

rescale_slope = image.GetMetaData('RescaleSlope')
rescale_intercept = image.GetMetaData('RescaleIntercept')

In [ ]:
import os
import pydicom
import numpy as np
import glob
# Specify the path to the folder containing DICOM slices
folder_path = '/home/abdullah/Downloads/archive (1)/FUMPE/CT_scans/PAT002'  # Replace with the actual folder path
dicom_paths=sorted(glob.glob(folder_path+"/*.dcm"),reverse=True)
images=[""]*(len(dicom_paths))
for dicom_path in dicom_paths:

    first_dicom = pydicom.dcmread(dicom_path)
    cx=first_dicom.pixel_array
    HU=cx*float(first_dicom.RescaleSlope)+float(first_dicom.RescaleIntercept)
    images[first_dicom.InstanceNumber-1]=HU

    


In [ ]:
volume_3d=np.dstack(images)

In [ ]:
from matplotlib import pyplot as plt
plt.figure(figsize=(26,10))
plt.imshow(volume_3d[20,:,:].squeeze(),cmap='gray')

In [ ]:
cx=first_dicom.pixel_array
HU=cx*float(first_dicom.RescaleSlope)+float(first_dicom.RescaleIntercept)

In [ ]:
HU.shape

In [ ]:
def view_images(files, title = '', aug = None, windowing = True):
    width = 2
    height = 5
    fig, axs = plt.subplots(height, width, figsize=(15,15))
    
    for im in range(0, height * width):
        #data = dcm.dcmread(files)
        image = files[:,:,im]
        window_center , window_width, intercept, slope = get_windowing(data)
        if windowing:
            output = window_image(image, window_center, window_width, intercept, slope, rescale = False)
        else:
            output = image
        i = im // width
        j = im % width
        axs[i,j].imshow(output, cmap=plt.cm.gray) 
        axs[i,j].axis('off')
        
    plt.suptitle(title)
    plt.show()
def window_image(img, window_center,window_width, intercept, slope, rescale=True,hu_conv=False):
    if hu_conv:
        img = (img*slope +intercept) #for translation adjustments given in the dicom file. 
    img_min = window_center - window_width//2 #minimum HU level
    img_max = window_center + window_width//2 #maximum HU level
    img=img*(img<img_max)*(img>img_min)
    
    print(img_min,img_max)

    if rescale: 
        img = (img - img_min) / (img_max - img_min)*255.0 
    return img


def get_first_of_dicom_field_as_int(x):
    #get x[0] as in int is x is a 'pydicom.multival.MultiValue', otherwise get int(x)
    if type(x) == dcm.multival.MultiValue: return int(x[0])
    else: return int(x)
def get_windowing(data):
    dicom_fields = {"window_center":data[('0028','1050')].value, #window center
                    "window_width":data[('0028','1051')].value, #window width
                    "intercept":data[('0028','1052')].value, #intercept
                    "slope":data[('0028','1053')].value} #slope
    return dicom_fields
window_dict={
'stroke':{'w':40,'l':40},
'temporal_bones':{'w':2800,'l':600},
'soft_tissues':{'w':350,'l':60},
'chest_lungs':{'w':1500,'l':-600},
'mediastinum':{'w':350,'l':50},
'abdomen_soft':{'w':400,'l':50},
'liver':{'w':150,'l':30},
'spine_soft':{'w':240,'l':50},
'bone':{'w':1800,'l':400}

}

window_dict['stroke']['w']

In [ ]:
kwargs=get_windowing(first_dicom)
kwargs['window_center']=400
kwargs['window_width']=1800

bone=window_image(volume_3d[220,:,:],**kwargs,rescale=True)

kwargs=get_windowing(first_dicom)
kwargs['window_center']=40
kwargs['window_width']=350
brain=window_image(volume_3d[220,:,:],**kwargs,rescale=True)
image=brain*(1-(bone>500))



In [ ]:
bone.max()

In [ ]:
kwargs

In [ ]:

from matplotlib import pyplot as plt
plt.figure(figsize=(26,10))
imax=first_dicom.pixel_array


plt.imshow(brain.T,cmap='gray')


In [ ]:
import sys
sys.path.append("..")
from segment_anything import sam_model_registry, SamAutomaticMaskGenerator, SamPredictor

sam_checkpoint = "/home/abdullah/BOUN/awesome/segment-anything/sam_vit_h_4b8939.pth"
model_type = "vit_h"

device = "cuda"

sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)

mask_generator = SamAutomaticMaskGenerator(sam)

In [ ]:
image_array = sitk.GetArrayFromImage(image_gantry)
image_array.shape

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(image_array[255,:,:])

In [ ]:
masks=[]
from tqdm import tqdm
for image in tqdm(image_array.transpose(1,2,0)):

    masks.append(mask_generator.generate(np.dstack([image,image,image]).astype('uint8')))

In [ ]:
mask_generator.generate(image.astype('uint8'))


In [ ]:
np.stack([image_array.transpose(1,2,0),image_array.transpose(1,2,0),image_array.transpose(1,2,0)],axis=-1).shape

In [ ]:
image_array.transpose(1,2,0).shape

In [ ]:
def show_anns(anns):
    if len(anns) == 0:
        return
    sorted_anns = sorted(anns, key=(lambda x: x['area']), reverse=True)
    ax = plt.gca()
    ax.set_autoscale_on(False)
    polygons = []
    color = []
    for ann in sorted_anns:
        m = ann['segmentation']
        img = np.ones((m.shape[0], m.shape[1], 3))
        color_mask = np.random.random((1, 3)).tolist()[0]
        for i in range(3):
            img[:,:,i] = color_mask[i]
        ax.imshow(np.dstack((img, m*0.35)))

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(image.T)
show_anns(masks)
plt.axis('off')
plt.show() 

In [ ]:
import SimpleITK as sitk

# Load the CT angiography image
image_path = "1.2.410.200010.20230508.145831.100041.103722_Beyin+Boyun_Anjio_Head_20230504005400_703.nii"
image = sitk.ReadImage(image_path)

# Preprocess the image (optional)
# You can apply noise reduction, image resampling, or other preprocessing steps here

# Vessel segmentation
threshold = 100  # Adjust the threshold value as needed
binary_image = image > threshold


# Perform post-processing (optional)
# You can apply morphological operations or other filters to refine the segmented vessels

# Calculate vessel volume
voxel_volume = image.GetSpacing()[0] * image.GetSpacing()[1] * image.GetSpacing()[2]  # Calculate voxel volume
vessel_volume = sitk.GetArrayFromImage(binary_image).sum() * voxel_volume

# Display the calculated vessel volume
print("Vessel Volume (mm^3):", vessel_volume)


In [ ]:
array = sitk.GetArrayFromImage(binary_image)

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(array[:,400,:])

In [ ]:
import pydicom
import numpy as np
import glob
# Load the DICOM image
paths=glob.glob("*/*")


for path in paths:
    try:
        dicom = pydicom.dcmread(glob.glob(path+"/*.dcm")[0])

        # Get the direction cosines
        direction_cosines = np.array(dicom.ImageOrientationPatient).reshape(2, 3)

        # Define the axis names for the three orientations
        axis_names = ['Sagittal', 'Coronal', 'Axial']

        # Calculate the dot product between the direction cosines and the three axes
        dot_products = np.abs(direction_cosines.dot(np.eye(3)))

        # Determine the dominant axis based on the maximum dot product
        dominant_axis = np.argmax(dot_products)

        # Print the dominant axis and corresponding orientation
        print('Dominant Axis:', axis_names[dominant_axis])
    except:
        pass
